In [11]:
import pandas as pd; import numpy as np; from shapely import geometry; import geopandas as gpd
import json
%matplotlib inline

# Breaking everything down to Buildings

In [12]:
buildings = pd.read_csv('../raw_data/ZayoHackathonData_Buildings.csv')
sites     = pd.read_csv('../raw_data/ZayoHackathonData_Sites.csv')
accounts  = pd.read_csv('../raw_data/ZayoHackathonData_Accounts.csv')
opportunities = pd.read_csv('../raw_data/ZayoHackathonData_Opportunities.csv')

In [13]:
print(buildings.columns)
buildings.head(1)

Index(['Building ID', 'Market', 'Street Address', 'City', 'State',
       'Postal Code', 'Latitude', 'Longitude', 'On Zayo Network Status',
       'Net Classification', 'Type', 'Network Proximity',
       ' Estimated Build Cost '],
      dtype='object')


,Building ID,Market,Street Address,City,State,Postal Code,Latitude,Longitude,On Zayo Network Status,Net Classification,Type,Network Proximity,Estimated Build Cost
0,Bldg-115472,Atlanta,953 Donnelly Ave SW,Atlanta,GA,30310,33.728521,-84.418751,Not on Zayo Network,Fiber Only,Office - Multi Tenant,116.91,"$13,507.30"


In [14]:
buildings['est_cost'] = buildings[' Estimated Build Cost '].replace('[\$,)]','', regex=True ).astype(float)

In [15]:
buildings.head(3)

,Building ID,Market,Street Address,City,State,Postal Code,Latitude,Longitude,On Zayo Network Status,Net Classification,Type,Network Proximity,Estimated Build Cost,est_cost
0,Bldg-115472,Atlanta,953 Donnelly Ave SW,Atlanta,GA,30310,33.728521,-84.418751,Not on Zayo Network,Fiber Only,Office - Multi Tenant,116.91,"$13,507.30",13507.30
1,Bldg-115473,Atlanta,555 Old Norcross Rd,Lawrenceville,GA,30046,33.946700,-83.995000,Not on Zayo Network,Fiber Only,Office - Multi Tenant,28323.03,"$689,752.72",689752.72
2,Bldg-115475,Atlanta,3159 Royal Dr,Alpharetta,GA,30005,34.056608,-84.265246,On Zayo Network,Fiber Only,Office - Multi Tenant,979.44,"$33,506.56",33506.56


In [16]:
sites.head(1)

,Site ID,Account ID,Building ID,Market,Site Address,City,State,Postal Code,Network Proximity,On Zayo Network Status
0,Site-000001,Acct-000025,Bldg-115472,Atlanta,953 Donnelly Ave SW,Atlanta,GA,30310,116.91,Not on Zayo Network


In [17]:
accounts.head(1)

,Account ID,Industry,Vertical,Total BRR,AnnualRevenue,NumberOfEmployees,DandB Revenue,DandB Total Employees
0,Acct-000273,Telecommunications,Wireless,"$6,084,906.00","$248,747,365.62",397,"$356,943,618.45",397


In [18]:
opportunities.head(1)

,Opportunity ID,Account ID,StageName,IsClosed,IsWon,CreatedDate,Term in Months,Service,Opportunity Type,Product Group,Building ID,Market,Street Address,City,State,Postal Code,Network Proximity,On Zayo Network Status
0,Opp-000001,Acct-000007,Closed - Lost,True,False,7/1/16,12.0,NaN,New Service,Ethernet,Bldg-012582,Atlanta,56 Marietta St NW,Atlanta,GA,30303,66.45,On Zayo Network


In [19]:
df = accounts.merge(sites,on="Account ID")

In [20]:
df2 = df.merge(buildings, on="Building ID")

In [21]:
df2.head(1)

,Account ID,Industry,Vertical,Total BRR,AnnualRevenue,NumberOfEmployees,DandB Revenue,DandB Total Employees,Site ID,Building ID,...,State_y,Postal Code_y,Latitude,Longitude,On Zayo Network Status_y,Net Classification,Type,Network Proximity_y,Estimated Build Cost,est_cost
0,Acct-000273,Telecommunications,Wireless,"$6,084,906.00","$248,747,365.62",397,"$356,943,618.45",397,Site-000061,Bldg-115571,...,GA,30309,33.800922,-84.393502,On Zayo Network,Fiber Only,Office - Multi Tenant,198.92,"$15,967.60",15967.6


In [22]:
df2.head(2).columns

Index(['Account ID', 'Industry', 'Vertical', ' Total BRR ', ' AnnualRevenue ',
       'NumberOfEmployees', ' DandB Revenue ', 'DandB Total Employees',
       'Site ID', 'Building ID', 'Market_x', 'Site Address', 'City_x',
       'State_x', 'Postal Code_x', 'Network Proximity_x',
       'On Zayo Network Status_x', 'Market_y', 'Street Address', 'City_y',
       'State_y', 'Postal Code_y', 'Latitude', 'Longitude',
       'On Zayo Network Status_y', 'Net Classification', 'Type',
       'Network Proximity_y', ' Estimated Build Cost ', 'est_cost'],
      dtype='object')

In [23]:
opportunities.head(2)

,Opportunity ID,Account ID,StageName,IsClosed,IsWon,CreatedDate,Term in Months,Service,Opportunity Type,Product Group,Building ID,Market,Street Address,City,State,Postal Code,Network Proximity,On Zayo Network Status
0,Opp-000001,Acct-000007,Closed - Lost,True,False,7/1/16,12.0,NaN,New Service,Ethernet,Bldg-012582,Atlanta,56 Marietta St NW,Atlanta,GA,30303,66.45,On Zayo Network
1,Opp-000002,Acct-000986,5 - Accepted,True,True,7/1/16,60.0,NaN,New Service,Dark Fiber - Metro,Bldg-016855,Atlanta,300 Satellite Blvd NW,Suwanee,GA,30024,374.79,On Zayo Network


In [31]:
x = df2.groupby("Building ID").agg({
        "Latitude"  : {'lat': lambda x: x.values[0]},
        "Longitude" : {'lon': lambda x: x.values[0]},
        "Account ID": {"Number Accounts": pd.Series.nunique},
        "est_cost"  : {"BuildingCost" : sum},
        "Site ID" : {"Number Sites" : pd.Series.nunique},
        "On Zayo Network Status_y": {
            "netStat" : lambda x: x.values[0]=='On Zayo Network'}
    })
x.columns = x.columns.get_level_values(1)

In [32]:
opp_count = pd.DataFrame(opportunities.query('StageName!="Closed - Lost"')['Building ID'].value_counts())

In [33]:
opp_count.head()

,Building ID
Bldg-014866,417
Bldg-012582,175
Bldg-017035,148
Bldg-065056,94
Bldg-127057,76


In [34]:
gdf = gpd.GeoDataFrame(x)
gdf.geometry = gdf.apply(lambda row: geometry.Point(row.lon, row.lat), axis=1)

In [42]:
print(len(gdf))
gdf.head(2)

17209


,Number Sites,lon,Number Accounts,netStat,BuildingCost,lat,geometry,opportunity
Building ID,,,,,,,,
Bldg-011543,3,-104.860338,3,True,71994.9,39.598945,POINT (-104.860338 39.598945),0
Bldg-012582,286,-84.391561,275,True,3430141.0,33.755431,POINT (-84.391561 33.755431),0


## Add opportunities (lookup)

In [43]:
gdf['opportunity'] = 0

In [44]:
gdf.head(2)

,Number Sites,lon,Number Accounts,netStat,BuildingCost,lat,geometry,opportunity
Building ID,,,,,,,,
Bldg-011543,3,-104.860338,3,True,71994.9,39.598945,POINT (-104.860338 39.598945),0
Bldg-012582,286,-84.391561,275,True,3430141.0,33.755431,POINT (-84.391561 33.755431),0


In [51]:
def make_geojson(row):
    if(row["netStat"]):
        cost = 0;
    else:
        cost = row["BuildingCost"]
    feat = {"type":"Feature",
            "properties":{
                "sites"       : row["Number Sites"],
                "accounts"    : row["Number Accounts"],
                "cost"        : cost,
                "opportunity" : row["opportunity"]
            },
            "geometry":{"type":"Point","coordinates":[]}}
    if (not np.isnan(row.lon) and not np.isnan(row.lat)):
        feat['geometry']['coordinates'].append(row.lon)
        feat['geometry']['coordinates'].append(row.lat)
    
        return feat

In [52]:
#Create GeoJSON
features = []
for idx,row in gdf[~gdf.netStat].iterrows():
    r = make_geojson(row)
    if r:
        features.append(r)
with open('../docs/data/offNetwork.geojson','w') as oFile:
    json.dump({"type":"FeatureCollection","features":features},oFile,indent=4)

In [53]:
#Create GeoJSON
features = []
for idx,row in gdf[gdf.netStat].iterrows():
    r = make_geojson(row)
    if r:
        features.append(r)
with open('../docs/data/onNetwork.geojson','w') as oFile:
    json.dump({"type":"FeatureCollection","features":features},oFile,indent=4)